<img src="http://meusite.mackenzie.br/rogerio/mackenzie_logo/UPM.2_horizontal_vermelho.jpg"  width=300, align="right">
<br>
<br>
<br>
<br>
<br>

# ***Título do seu projeto***
---

*Siga esse template para entrega do seu projeto. O código deste notebook precisa ser 100% executável.*

In [37]:
#@title Identificação do Grupo

#@markdown Turma **7N**

#@markdown Integrantes do Grupo (*informe \<TIA\>,\<nome\>*)
Aluno1 = '31928900, Ítalo Assunção' #@param {type:"string"} 
Aluno2 = '31986773, Rafael Alencar' #@param {type:"string"}
Aluno3 = '31617212, Felipe Retondar' #@param {type:"string"}
Aluno4 = '31946968, Roberto Tsuneki' #@param {type:"string"}


# Apresentação do Trabalho

#### A proposta do trabalho é predizer se uma pessoa vai ter uma doença cardíaca, baseado nos dados: 


- BMI - Índice de Massa Corpórea (float)
- Smoking - Fumou pelo menos 100 cigarros durante a vida toda (sim/não)
- AlcoholDrinking - Homem adulto que bebe mais de 14 drinks por semana ou Mulher adulta que bebe mais de 7 drinks por semana (sim/não)
- Stroke - Já teve um derrame (sim/não)
- PhysicalHealth - Quantas vezes nos últimos 30 dias a saúde física (doenças e machucados) não foi boa (int)
- MentalHealth - Quantas vezes nos últimos 30 dias a saúde mental não foi boa (int)
- DiffWalking - Dificuldade em andar (sim/não)
- Sex - Sexo da pessoa (homem/mulher)
- AgeCategory - Idade (intervalado)
- Race - Raça (branco/hispanico/negro/asiático/outro)
- Diabetic - É diabético (sim/não)
- PhysicalActivity - Fez atividade física (sem contar trabalho) nos últimos 30 dias (sim/não)
- GenHealth - Quão boa é a saúde geral (pobre/razoavel/boa/muito boa/excelente)
- SleepTime - Horas de sono por dia em média (int)
- Asthma - Tem/teve asma (sim/não)
- KidneyDisease - Tem/teve doença renal (sem contar pedra no rim, infecção na bexiga e incontinência)
- SkinCancer - Tem/teve câncer de pele (sim/não)

##### *O dataset pode ser encontrado neste [link](https://www.kaggle.com/datasets/kamilpytlak/personal-key-indicators-of-heart-disease)*

# Referencial Teórico

Foram usados os materiais vistos em aula, bem como as discussões, documentações e vídeos dos links abaixo:

- [Documentação SciKit Learn](https://scikit-learn.org)
- [Documentação Pandas](https://pandas.pydata.org/docs/reference/index.html#api)
- [Documentação NumPy](https://numpy.org/doc/)

# Base de Dados 

#### Foram usados todos os dados citados na apresentação, pois todos são considerados fatores chaves para doenças cardíacas. Foram necessárias algumas alterações nos dados em texto para dados numéricos, conforme explicado abaixo.

In [62]:
import pandas as pd
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
import numpy as np
import warnings

pd.set_option('display.max_columns', None)
warnings.filterwarnings("ignore")

In [24]:
raw = pd.read_csv('heart_2020_cleaned.csv')

raw.head()

,HeartDisease,BMI,Smoking,AlcoholDrinking,Stroke,PhysicalHealth,MentalHealth,DiffWalking,Sex,AgeCategory,Race,Diabetic,PhysicalActivity,GenHealth,SleepTime,Asthma,KidneyDisease,SkinCancer
0,No,16.60,Yes,No,No,3.0,30.0,No,Female,55-59,White,Yes,Yes,Very good,5.0,Yes,No,Yes
1,No,20.34,No,No,Yes,0.0,0.0,No,Female,80 or older,White,No,Yes,Very good,7.0,No,No,No
2,No,26.58,Yes,No,No,20.0,30.0,No,Male,65-69,White,Yes,Yes,Fair,8.0,Yes,No,No
3,No,24.21,No,No,No,0.0,0.0,No,Female,75-79,White,No,No,Good,6.0,No,No,Yes
4,No,23.71,No,No,No,28.0,0.0,Yes,Female,40-44,White,No,Yes,Very good,8.0,No,No,No


##### Precisamos transformar os dados não numérico em numéricos, e para isso usaremos 2 métodos:

- Se o dado for binário (Yes/No), transformamos em 0 (No) e 1 (Yes)
- Se o dado for texto ou um interval (GenHealth, AgeCategory etc) aplicamos hot encode

In [25]:
# Removendo linhas com valores nulos
raw = raw.dropna()

# Função para tornar Yes/No em 1/0
def binarify(x):
  if (x == 'Yes'):
    return 1
  
  if (x == 'No'):
    return 0
  
  return x

# Aplicando função para casos de Yes/No
raw['HeartDisease'] = raw['HeartDisease'].apply(binarify)

raw['Smoking'] = raw['Smoking'].apply(binarify)
raw['AlcoholDrinking'] = raw['AlcoholDrinking'].apply(binarify)
raw['Stroke'] = raw['Stroke'].apply(binarify)
raw['DiffWalking'] = raw['DiffWalking'].apply(binarify)
raw['Diabetic'] = raw['Diabetic'].apply(binarify)
raw['PhysicalActivity'] = raw['PhysicalActivity'].apply(binarify)
raw['Asthma'] = raw['Asthma'].apply(binarify)
raw['KidneyDisease'] = raw['KidneyDisease'].apply(binarify)
raw['SkinCancer'] = raw['SkinCancer'].apply(binarify)

# Aplicando hot encode para casos de textos/intervalos
dummies = pd.get_dummies(raw.Sex, prefix='Sex')
raw = pd.concat([raw, dummies], axis=1)
dummies = pd.get_dummies(raw.AgeCategory, prefix='AgeCategory')
raw = pd.concat([raw, dummies], axis=1)
dummies = pd.get_dummies(raw.Race, prefix='Race')
raw = pd.concat([raw, dummies], axis=1)
dummies = pd.get_dummies(raw.GenHealth, prefix='GenHealth')
raw = pd.concat([raw, dummies], axis=1)

# Removendo as colunas em que aplicamos hot encode
df = raw.drop(columns=['Sex', 'AgeCategory', 'Race', 'GenHealth'])

# Separando dataset em 70% para treino, 30% para teste
x_train, x_test, y_train, y_test = train_test_split(df.drop(columns='HeartDisease'), df.HeartDisease, train_size=0.7)

# df.head()

1


# Modelo de Regressão Linear

#### Variáveis Dependentes:
- HeartDisease

#### Variáveis Independentes:
- BMI
- Smoking
- AlcoholDrinking
- Stroke
- PhysicalHealth
- MentalHealth
- DiffWalking
- Sex
- AgeCategory
- Race
- Diabetic
- PhysicalActivity
- GenHealth
- SleepTime
- Asthma
- KidneyDisease
- SkinCancer

#### Transformações

Todas as transformações necessárias foram descritas acima.

In [68]:
model = linear_model.LinearRegression()

model.fit(x_train, y_train)

pred = model.predict(x_test)

# Transformando os dados em binário, considerando o threshold de >= 0.5 como 1 (tem doença cardíaca) e < 0.5  como 0 (não tem doença cardíaca)
pred = [1 if x >= 0.5 else 0 for x in pred]

# Verificando precisão do modelo
cnf_matrix = confusion_matrix(y_test, pred)

verdadeiro_positivo = cnf_matrix[0][0]
falso_positivo = cnf_matrix[0][1]
falso_negativo = cnf_matrix[1][0]
verdadeiro_negativo = cnf_matrix[1][1]

print(f'Acertos: {verdadeiro_negativo + verdadeiro_positivo}')
print(f'Erros: {falso_negativo + falso_positivo}')
print(f'Precisão: {accuracy_score(y_test, pred)}')

Acertos: 87805
Erros: 8134
Precisão: 0.9152169607771605


## Análise do Modelo Linear

O modelo linear não é intuitivo nem o ideal quando se trata de modelos de classificação. Contudo, sua aplicação é possível de ser feita, sendo necessário, porém, realizar hot encode para transformar dados não-numéricos em dados numéricos, aplicar o modelo e, então, em sua predição criar um threshold (intuitivamente 0.5) para "binarizar" o resultado em 0 e 1.

Isso causa uma perda de precisão e deixa o modelo sensível na adição de novos dados, tornando-se necessário o treinamento a cada nova adição e a revalidação dos thresholds.

## Discussão dos Resultados do Modelo Linear

Como pudemos ver pela precisão do modelo, embora não seja o modelo mais ideal para classificação, ele tem uma considerável taxa de acertividade com aproximadamente 91.52% **neste caso**.

# Modelo Logístico

#### Variáveis Dependentes:
- HeartDisease

#### Variáveis Independentes:
- BMI
- Smoking
- AlcoholDrinking
- Stroke
- PhysicalHealth
- MentalHealth
- DiffWalking
- Sex
- AgeCategory
- Race
- Diabetic
- PhysicalActivity
- GenHealth
- SleepTime
- Asthma
- KidneyDisease
- SkinCancer

#### Transformações

Todas as transformações necessárias foram descritas acima.

In [67]:
model2 = linear_model.LogisticRegression()

model2.fit(x_train, y_train)

pred2 = model2.predict(x_test)

cnf_matrix2 = confusion_matrix(y_test, pred2)

verdadeiro_positivo2 = cnf_matrix2[0][0]
falso_positivo2 = cnf_matrix2[0][1]
falso_negativo2 = cnf_matrix2[1][0]
verdadeiro_negativo2 = cnf_matrix2[1][1]

print(f'Acertos: {verdadeiro_negativo2 + verdadeiro_positivo2}')
print(f'Erros: {falso_negativo2 + falso_positivo2}')
print(f'Precisão: {accuracy_score(y_test, pred2)}')

Acertos: 87813
Erros: 8126
Precisão: 0.9153003470955503


## Análise do Modelo Logístico

O Modelo Logístico é o modelo mais indicado (quando comparado com o modelo linear) para problemas de classificação.

Também encontramos a necessidade de utilizar hot encode para transformar os dados.

## Discussão dos Resultados do Modelo Logístico

Conforme pode-se ver no resultado da precisão, o modelo logístico conseguiu chegar num resultado considerável de aproximadamente 91.53% de acertividade **neste caso**


# Conclusão 

Comparando os dois modelos, é perceptível a diferença de implementação: o modelo linear requer a transformação do resultado predito em valor binário (para se qualificar como modelo de classificação), enquanto este resultado já é o retornado pelo modelo logístico, escancarando a preferência desde caso de uso.

Sabendo disso, entretanto, os resultados se mostraram extremamente próximos, havendo uma diferença mínima desconsiderável de 1/10000 para os dados utilizados neste estudo. 

# Referências 

Além do Referencial Teórico, também foram utilizados esses links:

- [Dataset - Kaggle](https://www.kaggle.com/datasets/kamilpytlak/personal-key-indicators-of-heart-disease)
- [Para entender como classificar binariamente usando Regressão Linear](https://discuss.analyticsvidhya.com/t/using-linear-regression-for-a-classification-problem/9848/3)
- [Porque não usar Regressão Linear para modelos de classificação](https://www.youtube.com/watch?v=2TvKZnTHC4M)
- [Porque a Regressão Linear é pior do que outros modelos de classificação](https://stats.stackexchange.com/a/22382)

---

In [60]:
#@title Avaliação
Completo = 10 #@param {type:"slider", min:0, max:10, step:1}
#@markdown Projeto cumpre todos os itens pedidos. 
Relevancia = 9 #@param {type:"slider", min:0, max:10, step:1}
#@markdown As seleções de dados e eventos para análise são relevantes e justificados. 
Tecnicas = 6 #@param {type:"slider", min:0, max:10, step:1}
#@markdown As técnicas de empregadas são adequadas e corretamente aplicadas.
Apresentacao = 7 #@param {type:"slider", min:0, max:10, step:1}
#@markdown A apresentação dos resultados é clara e objetiva.
Analise = 8 #@param {type:"slider", min:0, max:10, step:1}
#@markdown As premissas de análise se justificam e a analise é correta. 
Conclusao = 7 #@param {type:"slider", min:0, max:10, step:1}
#@markdown As conclusões são justificadas e relevantes
Bonus = 0.5 #@param {type:"slider", min:0, max:1, step:0.5}
#@markdown A critério do professor por inovações na abordagem e no uso de técnicas de Análise de Dados








In [ ]:
#@markdown ### Nota Final
nota = Completo + Relevancia + Tecnicas + Apresentacao + Analise + Conclusao 

nota = nota / 6 + Bonus

print(f'Nota final do trabalho {nota :.1f}')

import numpy as np
import pandas as pd

alunos = pd.DataFrame()

lista_tia = []
lista_nome = []

for i in range(1,6):
  exec("if Aluno" + str(i) + " !='None':  lista = Aluno" + str(i) + ".split(','); lista_tia.append(lista[0]); lista_nome.append(lista[1].upper())")

alunos['tia'] = lista_tia
alunos['nome'] = lista_nome
alunos['nota'] = np.round(nota,1)
print()
alunos

Nota final do trabalho 8.3



,tia,nome,nota
0,1115665,ADRIANA FUJITA,8.3
1,1115677,DANIEL HENRIQUE,8.3
